In [2]:
import re
import json

import urllib3
import configparser

import tensorflow_hub as hub
import tensorflow.compat.v1 as tf

from tqdm import tqdm
from bson import InvalidBSON
from pymongo import MongoClient
from bson.json_util import dumps
from elasticsearch import Elasticsearch

In [3]:
print('Downloading pre-trained embeddings from tensorflow hub...')
tf.disable_eager_execution()
embed = hub.Module('../out/models/universal-sentence-encoder')
text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)
print('Done.')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Done.


In [4]:
print('Creating tensorflow session...')
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
print('Done.')

Creating tensorflow session...


2023-11-20 19:07:11.441386: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-20 19:07:12.362873: W tensorflow/c/c_api.cc:305] Operation '{name:'module/string_to_index_11/hash_table/table_init' id:3918 op device:{requested: '', assigned: ''} def:{{{node module/string_to_index_11/hash_table/table_init}} = InitializeTableV2[Tkey=DT_STRING, Tval=DT_INT64, _has_manual_control_dependencies=true](module/string_to_index_11/hash_table, module/Embeddings_en_words, module/string_to_index_11/ToInt64)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Done.


In [5]:
urllib3.disable_warnings()

config = configparser.ConfigParser()
config.read('../server.ini');

In [6]:
# Elastic
es = Elasticsearch(
    config['ELASTIC']['path'],
    verify_certs=False
);

/opt/homebrew/Caskroom/miniconda/base/envs/thesis/lib/python3.10/site-packages/elasticsearch/connection/http_urllib3.py:209: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [7]:
# Mongo
client = MongoClient(config['MONGODB']['path'])
swagger = client['swagger']['apis']

In [48]:
tot = 0
skipped = 0

batch = swagger.find(batch_size=100)
count = batch.count()

for n in tqdm(range(230840, count)):
    try:
        document = json.loads(dumps(batch.skip(n).limit(1)[0]))
        nl_tags = r'(?:\'|\")(?:description|name|title|summary)(?:\'|\"):\s(?:\'|\")([^\'\"]+)(?:\'|\")'

        obj = {
            'id': str(document['_id']['$oid']),
            'url': str(document['_API_reference']),
            'name': str(document['_name']),
            'version': str(document['_version']) if not str(document['_version'].startswith('v')) else str(document['_version'][1:]),
            'embedding': session.run(embeddings, feed_dict={text_ph: [' '.join(re.findall(nl_tags, str(document)))]}).tolist()[0]
        }

        es.index(index='migration-test', body=obj)
        
        tot += 1
    except InvalidBSON:
        skipped += 1
    except TypeError:
        skipped += 1

print(tot, '/', count)
print(skipped, '/', count)

/var/folders/yb/m5pxnktn1l94k6xgffj6j1y00000gn/T/ipykernel_4216/2905656582.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  count = batch.count()
  0%|          | 4/201429 [00:10<152:51:31,  2.73s/it]


KeyboardInterrupt: 

In [26]:
query = 'eating healthy'

knn_query = {
    'knn': {
        'field': 'embedding',
        'query_vector': session.run(embeddings, feed_dict={text_ph: [query]}).tolist()[0],
        'k': 5,
        'num_candidates': 10000
    }
}

results = es.search(index='migration-test', body=knn_query, _source=['name', 'version'])['hits']['hits']

In [27]:
print(f'These are the top 5 results of the query "{query}":\n')

for ind, result in enumerate(results):
    length_name = max([len(el['_source']['name']) for el in results]) + 2
    length_version = max([len(el['_source']['version']) for el in results]) + 2

    print(
        f'\t{str(ind + 1) + ".": <2} {result["_source"]["name"]: <{length_name}} v.{result["_source"]["version"]: <{length_version}} [{str(result["_score"] * 100)[:2]}%]')

These are the top 5 results of the query "eating healthy":

	1. fat_free_title   v..0.1   [74%]
	2. eating           v..0.0   [73%]
	3. rest ibs         v.1      [72%]
	4. IBS for MK       v.       [69%]
	5. ATPJWT-api       v..0.1   [68%]
